In [1]:
# Cargar variables de entorno desde .env
from dotenv import load_dotenv
import os
load_dotenv()  # Esto cargará las variables definidas en el archivo .env

False

# Construir un Chatbot

# **Descripción general**

Revisaremos un ejemplo de cómo diseñar e implementar un chatbot con motor LLM. Este chatbot podrá tener una conversación y recordar interacciones anteriores con un [modelo de chat](https://python.langchain.com/docs/concepts/chat_models/) .

Tenga en cuenta que este chatbot que construiremos solo usará el modelo del lenguaje para tener una conversación. Hay varios otros conceptos relacionados que puede estar buscando:

- [Conversación RAG](https://python.langchain.com/docs/tutorials/qa_chat_history/): Habilitar una experiencia de chatbot en la que se pueda conectar con una fuente externa de datos para producir respuestas basadas en ella.
- [Agentes](https://python.langchain.com/docs/tutorials/agents/) : Construir un chatbot que pueda llevar a cabo acciones.

Este tutorial cubrirá los conceptos básicos que serán útiles para esos dos temas más avanzados.

# **Instalación**

Para este tutorial necesitaremos `langchain-core` y `langgraph`. Esta guía requiere `langgraph >= 0.2.28`.

In [2]:
%pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.




# **LangSmith**

Muchas de las aplicaciones que construye con Langchain contendrán múltiples pasos con múltiples invocaciones de llamadas LLM. A medida que estas aplicaciones se vuelven cada vez más complejas, se vuelve crucial poder inspeccionar lo que está sucediendo exactamente dentro de su cadena o agente. La mejor manera de hacer esto es con [LangSmith](https://smith.langchain.com/).

Después de registrarse en el enlace de arriba, asegúrese de establecer sus variables de entorno para comenzar a registrar las trazas:




```
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
export LANGSMITH_PROJECT="default" # or any other project name
```



O, si en un cuaderno, puede configurarlos con:




In [3]:
import getpass
import os
try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass
os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "pr-glossy-thought-54"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "pr-glossy-thought-54"



# **Inicio rápido**

Primero, aprendamos a usar un modelo de idioma por sí mismo. Langchain admite muchos modelos de idioma diferentes que puede usar indistintamente: ¡seleccione el que desea usar a continuación!




## ¿Cómo reiniciar el kernel en Jupyter Notebook o JupyterLab?

Para reiniciar el kernel y limpiar la memoria de tu notebook, sigue estos pasos:

1. Ve al menú superior y selecciona: **Kernel** → **Restart Kernel** (o en español: **Núcleo** → **Reiniciar núcleo**).

2. Confirma el reinicio si se solicita.
3. Después del reinicio, ejecuta nuevamente las celdas desde el inicio para recargar todas las variables y librerías.

> No existe un comando de Python que reinicie el kernel y continúe automáticamente la ejecución de las celdas. El reinicio siempre requiere confirmación manual para evitar la pérdida accidental de datos en memoria.

In [9]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [12]:
import getpass
import os
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: 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 ")
from langchain.chat_models import init_chat_model
model = init_chat_model("llama3-8b-8192", model_provider="groq")



Primero usemos el modelo directamente. Los [ChatModels](https://python.langchain.com/docs/concepts/chat_models/) son instancias de [Runnables](https://python.langchain.com/docs/concepts/runnables/) de LangChain, lo que significa que exponen una interfaz estándar para interactuar con ellos. Para simplemente llamar al modelo, podemos pasar en una lista de [mensajes](https://python.langchain.com/docs/concepts/messages/) hacia el método `.invoke`.




In [14]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi! I'm Bob")])

APIConnectionError: Connection error.

> **Nota:** Antes de ejecutar la siguiente celda, asegúrate de haber ejecutado la celda donde se inicializa la variable `model`. Si no lo haces, obtendrás un error `NameError: name 'model' is not defined`.



**Referencia de API:** [HumanMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html)






El modelo por sí solo no tiene ningún concepto de estado. Por ejemplo, si hace una pregunta de seguimiento:




In [10]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't know your name. I'm a large language model, I don't have the ability to know or remember the names of individual people. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to tell me your name, I'd be happy to learn it!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 15, 'total_tokens': 92, 'completion_time': 0.182757874, 'prompt_time': 0.006508727, 'queue_time': 0.14326787100000002, 'total_time': 0.189266601}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--620ab153-f163-4b44-9439-0d7491ffab91-0', usage_metadata={'input_tokens': 15, 'output_tokens': 77, 'total_tokens': 92})



Echemos un vistazo [Trace de LangSmith](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r) del ejemplo.

Podemos ver que no lleva la conversación anterior como contexto y no puede responder la pregunta. ¡Esto lo convierte en una terrible experiencia en chatbot!

Para evitar esto, necesitamos pasar todo el [historial de conversación](https://python.langchain.com/docs/concepts/chat_history/) al modelo. Veamos qué sucede cuando hacemos eso:




In [11]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='I know this one! Your name is Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 40, 'total_tokens': 51, 'completion_time': 0.03776128, 'prompt_time': 0.024840335, 'queue_time': 0.143622949, 'total_time': 0.062601615}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e70a1e35-70fd-4084-83c6-c6dce08cb1d2-0', usage_metadata={'input_tokens': 40, 'output_tokens': 11, 'total_tokens': 51})

**Referencia de API:** [AIMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html)



¡Y ahora podemos ver que obtenemos una buena respuesta!

Esta es la idea básica que sustenta la capacidad de un chatbot para interactuar con conversación. Entonces, ¿cómo implementamos mejor esto?

# **Persistencia del mensaje**

[LangGraph](https://langchain-ai.github.io/langgraph/) Implementa una capa de persistencia incorporada, lo que lo hace ideal para aplicaciones de chat que admiten múltiples giros conversacionales.

Envolver nuestro modelo de chat en una aplicación mínima de LangGraph nos permite persistir automáticamente el historial de mensajes, simplificando el desarrollo de aplicaciones múltiples.

LangGraph viene con un simple checkpointer en memoria, que usamos a continuación. Vea su [documentación](https://langchain-ai.github.io/langgraph/concepts/persistence/) Para obtener más detalles, incluyendo cómo usar diferentes backends de persistencia (por ejemplo, SQLite o Postgres).




In [12]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Definir un nuevo grafo
workflow = StateGraph(state_schema=MessagesState)

# Definir la función que llama al modelo
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Definir el nodo (sólo uno) en el grafo
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Añadir memoria
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)



**Referencia de API:** [MemorySaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.memory.MemorySaver) | [StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.state.StateGraph)

Ahora necesitamos crear una `config`que pasaremos al runnable cada vez. Esta configuración contiene información que no es parte de la entrada directamente, pero que sigue siendo útil. En este caso, queremos incluir un `thread_id`. Esto debería verse como:




In [14]:
config = {"configurable": {"thread_id": "abc123"}}



Esto nos permite admitir múltiples hilos de conversación con una sola aplicación, un requisito común cuando su aplicación tiene múltiples usuarios.

Luego podemos invocar la aplicación:




In [15]:
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # La salida contiene todos los mensajes en el state y sólo nos interesa el último

================================== Ai Message ==================================

Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [16]:
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me your name is Bob!




¡Excelente! Nuestro chatbot ahora recuerda cosas sobre nosotros. Si cambiamos la configuración para hacer referencia a una diferente `thread_id`, podemos ver que comienza la conversación fresca.




In [17]:
config = {"configurable": {"thread_id": "abc234"}}
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm just an AI, I don't have any information about your personal identity or name. I'm a new conversation every time you interact with me, so I don't have any prior knowledge or context about you. Would you like to introduce yourself?




Sin embargo, siempre podemos volver a la conversación original (ya que la estamos persistiendo en una base de datos)




In [18]:
config = {"configurable": {"thread_id": "abc123"}}
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!




¡Así es como podemos admitir un chatbot que tiene conversaciones con muchos usuarios!

Para el soporte de async, defina el nodo `call_model` para que sea una función `async` y use `.ainvoke` al invocar la aplicación con `await`:




In [19]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
config = {"configurable": {"thread_id": "abc123"}}
query = "Compose a haiku with my name."
input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

What a unique request!

Since I don't know your name, I'll assume it's "Luna" for now. If you'd like to share your actual name, feel free to do so!

Here's a haiku with the name "Luna":

Luna's gentle glow
Moonbeams dance upon her
Silent midnight sky




En este momento, todo lo que hemos hecho es agregar una capa de persistencia simple alrededor del modelo. Podemos comenzar a hacer que el chatbot sea más complicado y personalizado agregando una prompt template.

# **Prompt Templates**

Las [promp templates](https://python.langchain.com/docs/concepts/prompt_templates/) Ayudan a convertir la información del usuario en bruto en un formato con el que el LLM pueda trabajar. En este caso, la entrada de usuario sin procesar es solo un mensaje, que estamos pasando al LLM. Hagámoslo ahora un poco más complicado. Primero, agregemos un mensaje del sistema con algunas instrucciones personalizadas (pero aún tomemos mensajes como entrada). A continuación, agregaremos más información además de los mensajes.

Para agregar un mensaje de sistema, crearemos un `ChatPromptTemplate`. Utilizaremos `MessagesPlaceholder`para transmitir todos los mensajes.




In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un escritor del Siglo de Oro español. Todo lo que dices lo adornas con frases barrocas y palabras del siglo XVI",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)



**Referencia de API:** [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) | [MessagesPlaceholder](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html)

Ahora podemos actualizar nuestra aplicación para incorporar esta plantilla:




In [20]:
workflow = StateGraph(state_schema=MessagesState)
def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)



Invocamos la aplicación de la misma manera:




In [23]:
config = {"configurable": {"thread_id": "abc345"}}
query = "¿Podrías decirme lo que es un botijo?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Ay, gentil amigo! Un botijo, ¡oh pasatiempo divino! Es un instrumento de música, un juglar de sonoras estridencias, un compañero de fiestas y de bodas, un verdadero placer para los sentidos. Un botijo, ¡oh instrumento de la alegría!, es un membranófono, es decir, un objeto que produce sonido al ser golpeado con las manos, generalmente de manera rítmica. ¡Y qué música produce, amigo mío! ¡Un sonido dulce y alegre, que hace saltar las piernas y hacer reír el estómago! ¡Un botijo, ¡oh deleite de los sentidos!, es un verdadero compañero de fiestas y de juegos, que hace que el tiempo pase volando, ¡y nos haga olvidar los pesares y las penas!


In [39]:
query = "¿Y cómo mantiene el agua fría?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Amigo mío, la cuestión de cómo mantiene el agua fría el botijo es un misterio que ha sido objeto de estudio y debate entre los sabios y los curiosos de la edad. ¡Es como si el botijo tuviera un secreto, un truco, una magia para conservar la frescura del líquido que contiene!

Pero, según se cree, el botijo mantiene el agua fría gracias a su forma particular, a su textura y a su materia prima. ¡Es como si el barro mismo se esforzara por conservar la frescura del líquido! La forma curva del botijo, su capacidad para absorber y conservar el frío, su capacidad para mantener el líquido en contacto con la piel del bebedor, todo esto contribuye a mantener la agua fresca y refrescante.

Y, por supuesto, amigo mío, la elegancia y la gracia del botijo también juegan un papel importante. ¡Es como si el botijo fuera un ángel que nos trae la frescura, que nos hace sentir la vida como un río de frescura que fluye por d



¡Impresionante! Ahora hagamos que nuestro aviso sea un poco más complicado. Supongamos que la plantilla rápida ahora se parece a esto:




In [25]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente de programación experto en {language}. Devuelves sólamente código.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)



Tenga en cuenta que hemos agregado un nuevo input (`language`). Nuestra aplicación ahora tiene dos parámetros: la entrada `messages`y `language`. Deberíamos actualizar el estado de nuestra aplicación para reflejar esto:




In [26]:
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
workflow = StateGraph(state_schema=State)
def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)



**Referencia de API:** [BaseMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.base.BaseMessage.html) | [add_messages](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.message.add_messages)




In [27]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Crea una función que genere números primos."
language = "Python"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

```
def generate_primes(n):
    sieve = [True] * (n+1)
    for x in range(2, int(n**0.5) + 1):
        if sieve[x]: 
            for i in range(x * x, n+1, x): 
                sieve[i] = False
    return [p for p in range(2, n) if sieve[p]]
```


Tenga en cuenta que todo el estado está persistido, por lo que podemos omitir parámetros como `language` Si no se desean cambios:

In [28]:
query = "Añádele docstrings y typehints"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

```
def generate_primes(n: int) -> list[int]:
    """
    This function generates a list of prime numbers up to the given number n.

    Args:
        n (int): The upper limit for the prime numbers.

    Returns:
        list[int]: A list of prime numbers up to n.
    """
    sieve = [True] * (n+1)
    for x in range(2, int(n**0.5) + 1):
        if sieve[x]: 
            for i in range(x * x, n+1, x): 
                sieve[i] = False
    return [p for p in range(2, n) if sieve[p]]
```




Para ayudarlo a comprender lo que está sucediendo internamente, consulte este [Trace de LangSmith](https://smith.langchain.com/public/15bd8589-005c-4812-b9b9-23e74ba4c3c6/r).

# **Gestión del historial de conversación**

Un concepto importante para entender al construir chatbots es cómo administrar el historial de conversación. Si se deja sin administrar, la lista de mensajes crecerá ilimitadamente y potencialmente desbordará la ventana de contexto de la LLM. Por lo tanto, es importante agregar un paso que limite el tamaño de los mensajes que está pasando.

**Es importante destacar que querrá hacer esto *antes* del prompt template, pero *después* de cargar mensajes anteriores del historial de mensajes.**

Podemos hacer esto agregando un paso simple frente a la solicitud que modifica la clave `messages` adecuadamente, y luego envuelva esa nueva cadena en la clase de historial de mensajes.

Langchain viene con algunos “ayudantes” incorporados para [Administrar una lista de mensajes](https://python.langchain.com/docs/how_to/#messages) . En este caso usaremos el ayudante [trim_messages](https://python.langchain.com/docs/how_to/trim_messages/) para reducir cuántos mensajes estamos enviando al modelo. El recortador nos permite especificar cuántos tokens queremos mantener, junto con otros parámetros, como si siempre queremos mantener el mensaje del sistema o si queremos permitir mensajes parciales:




In [30]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\Dell\PyhtonIA\ML Langchain\.venv Langchain\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]



**Referencia de API:** [SystemMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.system.SystemMessage.html) | [trim_messages](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.utils.trim_messages.html)






Para usarlo en nuestra cadena, solo necesitamos ejecutar el recortador antes de pasar la entrada `messages` a nuestro prompt.




In [57]:
workflow = StateGraph(state_schema=State)

def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)



Ahora, si intentamos preguntarle al modelo nuestro nombre, no lo sabrá ya que recortamos esa parte del historial de chat:




In [58]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"
input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name, but I'm happy to help you with your coding needs!




Pero si preguntamos sobre la información que se encuentra dentro de los últimos mensajes, recuerda:




In [59]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"
input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked "whats 2 + 2"?




Si echas un vistazo a LangSmith, puedes ver exactamente lo que está sucediendo debajo del capó en el [Trace de Langsmith](https://smith.langchain.com/public/04402eaa-29e6-4bb1-aa91-885b730b6c21/r) .

# Streaming

Ahora tenemos un chatbot en funcionamiento. Sin embargo, una consideración verdaderamente importante para la experiencia de usuario de las aplicaciones de chatbot es el flujo de streaming. Los LLM a veces pueden tardar un tiempo en responder, por lo que para mejorar la experiencia del usuario, una cosa que hacen la mayoría de las aplicaciones es transmitir cada token a medida que se genera. Esto permite al usuario ver el progreso.

¡En realidad es muy fácil hacer esto!

Por defecto, usar `.stream` en nuestros pasos de aplicación de la aplicación Langgraph, en este caso, el solo paso de la respuesta del modelo. La configuración `stream_mode="messages"`nos permite transmitir tokens de salida en su lugar:




In [60]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "Python"
input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Here|'s| a| joke| for| you|:

|``|`
|public| class| J|oke| {
|   | public| static| void| main|(String|[]| args|)| {
|       | System|.out|.println|("|Why| don|'t| scientists| trust| atoms|?");
|       | System|.out|.println|("|Because| they| make| up| everything|!");
|   | }
|}
|```||



# **Siguientes pasos**

Ahora que comprende los conceptos básicos de cómo crear un chatbot en Langchain, algunos tutoriales más avanzados que le interesan son:

- [Conversational RAG](https://python.langchain.com/docs/tutorials/qa_chat_history/): Habilitar una experiencia de chatbot sobre una fuente externa de datos
- [Agentes](https://python.langchain.com/docs/tutorials/agents/) : Construir un chatbot que pueda tomar medidas

Si desea sumergirse más profundamente en los detalles, algunas cosas que vale la pena visitar es:

- [Streaming](https://python.langchain.com/docs/how_to/streaming/): la transmisión en flujo es *crucial* para aplicaciones de chat
- [Cómo agregar el historial de mensajes](https://python.langchain.com/docs/how_to/message_history/) : para una inmersión más profunda en todas las cosas relacionadas con el historial de mensajes
- [Cómo administrar el historial de mensajes grandes](https://python.langchain.com/docs/how_to/trim_messages/) : Más técnicas para administrar un gran historial de chat
- [LangGraph main docs](https://langchain-ai.github.io/langgraph/): para obtener más detalles sobre el edificio con Langgraph
